# A manager's analysis of an unspecified coffee chain in the United States

Questions that will be queried in this analysis:
* Where are our most valuable markets?
* Which products should we focus on?
* Are our markets meeting their goals?

# Prerequisits: Importing the necessary libraries, database, and checking for nulls

In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.io as pio
from plotly.subplots import make_subplots
import plotly.graph_objects as go
pio.renderers.default = "kaggle"
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/coffee-chain-sales-dataset/Coffee_Chain_Sales .csv


In [2]:
# Import the Database
df=pd.read_csv('/kaggle/input/coffee-chain-sales-dataset/Coffee_Chain_Sales .csv') #The whitespace is part of the name
df.head(10)

,Area Code,Cogs,DifferenceBetweenActualandTargetProfit,Date,Inventory Margin,Margin,Market_size,Market,Marketing,Product_line,...,Product,Profit,Sales,State,Target_cogs,Target_margin,Target_profit,Target_sales,Total_expenses,Type
0,303,51,-35,10/1/2012,503,71,Major Market,Central,46,Leaves,...,Lemon,-5,122,Colorado,30,60,30,90,76,Decaf
1,970,52,-24,10/1/2012,405,71,Major Market,Central,17,Leaves,...,Mint,26,123,Colorado,30,60,50,90,45,Decaf
2,409,43,-22,10/2/2012,419,64,Major Market,South,13,Leaves,...,Lemon,28,107,Texas,30,60,50,90,36,Decaf
3,850,38,-15,10/3/2012,871,56,Major Market,East,10,Leaves,...,Darjeeling,35,94,Florida,40,60,50,100,21,Regular
4,562,72,6,10/4/2012,650,110,Major Market,West,23,Leaves,...,Green Tea,56,182,California,20,60,50,80,54,Regular
5,712,0,-29,10/5/2012,430,43,Small Market,Central,0,Beans,...,Decaf Espresso,31,43,Iowa,0,60,60,60,12,Decaf
6,860,47,-29,10/6/2012,375,64,Small Market,East,15,Beans,...,Decaf Espresso,21,111,Connecticut,30,60,50,90,43,Decaf
7,918,27,-39,10/7/2012,859,39,Small Market,South,7,Beans,...,Decaf Irish Cream,21,66,Oklahoma,30,60,60,90,18,Decaf
8,775,31,-43,10/8/2012,1000,37,Small Market,West,9,Beans,...,Decaf Irish Cream,7,68,Nevada,30,60,50,90,30,Decaf
9,435,40,-23,10/9/2012,881,59,Small Market,West,11,Beans,...,Decaf Espresso,37,99,Utah,20,60,60,80,22,Decaf


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1062 entries, 0 to 1061
Data columns (total 21 columns):
 #   Column                                  Non-Null Count  Dtype 
---  ------                                  --------------  ----- 
 0   Area Code                               1062 non-null   int64 
 1   Cogs                                    1062 non-null   int64 
 2   DifferenceBetweenActualandTargetProfit  1062 non-null   int64 
 3   Date                                    1062 non-null   object
 4   Inventory Margin                        1062 non-null   int64 
 5   Margin                                  1062 non-null   int64 
 6   Market_size                             1062 non-null   object
 7   Market                                  1062 non-null   object
 8   Marketing                               1062 non-null   int64 
 9   Product_line                            1062 non-null   object
 10  Product_type                            1062 non-null   object
 11  Prod

In [4]:
# Check for nulls and extra whitespaces
print(df.isnull().sum())
df.columns = df.columns.str.strip()
print(df.columns.to_list())

Area Code                                 0
Cogs                                      0
DifferenceBetweenActualandTargetProfit    0
Date                                      0
Inventory Margin                          0
Margin                                    0
Market_size                               0
Market                                    0
Marketing                                 0
Product_line                              0
Product_type                              0
Product                                   0
Profit                                    0
Sales                                     0
State                                     0
Target_cogs                               0
Target_margin                             0
Target_profit                             0
Target_sales                              0
Total_expenses                            0
Type                                      0
dtype: int64
['Area Code', 'Cogs', 'DifferenceBetweenActualandTargetProfit',

# Qustion 1: Where are our most valuable markets?

In [5]:
# --- Group by State: Total Sales and Profit ---
salestate = df.groupby('State', as_index=False)['Sales'].sum().sort_values('Sales', ascending=False)
profstate = df.groupby('State', as_index=False)['Profit'].sum().sort_values('Profit', ascending=False)

# --- Side-by-Side Comparison of Sales vs Profit by State ---
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig1 = make_subplots(rows=1, cols=2, subplot_titles=["Total Sales by State", "Total Profit by State"])
fig1.add_trace(go.Bar(x=salestate['State'], y=salestate['Sales'], name='Sales'), row=1, col=1)
fig1.add_trace(go.Bar(x=profstate['State'], y=profstate['Profit'], name='Profit'), row=1, col=2)
fig1.update_layout(
    height=500, width=1000,
    title_text="Sales vs Profit by State",
    xaxis_tickangle=-45,
    xaxis2_tickangle=-45
)
fig1.show()

# --- Print Top Performing States ---
print(f"Top Sales State: {salestate.iloc[0]['State']} with ${salestate.iloc[0]['Sales']:.2f} in sales")
print(f"Top Profit State: {profstate.iloc[0]['State']} with ${profstate.iloc[0]['Profit']:.2f} in profit")

Top Sales State: California with $23032.00 in sales
Top Profit State: Illinois with $7380.00 in profit


In [6]:
# --- [1] Combined Bar Chart: Sales vs Profit by State ---
state_summary = df.groupby('State', as_index=False)[['Sales', 'Profit']].sum()
state_summary = state_summary.sort_values('Sales', ascending=False)

fig2_combined = go.Figure(data=[
    go.Bar(name='Sales', x=state_summary['State'], y=state_summary['Sales']),
    go.Bar(name='Profit', x=state_summary['State'], y=state_summary['Profit'])
])
fig2_combined.update_layout(
    barmode='group',
    title='Sales vs Profit by State (Grouped Bar Chart)',
    xaxis_title='State',
    yaxis_title='Amount ($)',
    xaxis_tickangle=-45,
    height=500,
    width=1000
)
fig2_combined.show()


# --- [2] Add Target Comparisons: Sales & Profit ---
df['Profit_vs_Target'] = df['Profit'] - df['Target_profit']
df['Sales_vs_Target'] = df['Sales'] - df['Target_sales']


# --- [3] Summarize Actual vs Target by State ---
state_perf = df.groupby('State', as_index=False)[[
    'Sales', 'Target_sales', 'Sales_vs_Target',
    'Profit', 'Target_profit', 'Profit_vs_Target'
]].sum()


# --- [4] Identify Underperforming States ---
# Criteria: High sales but profit below target
underperforming_states = state_perf[
    (state_perf['Sales'] > state_perf['Target_sales']) &
    (state_perf['Profit'] < state_perf['Target_profit'])
]

print("🚩 States with High Sales but Profit Below Target:")
print(underperforming_states[['State', 'Sales', 'Target_sales', 'Profit', 'Target_profit']].to_string(index=False))


# --- [5] Visualize Underperforming States ---
if not underperforming_states.empty:
    fig2a = px.bar(
        underperforming_states,
        x='State',
        y=['Sales', 'Target_sales', 'Profit', 'Target_profit'],
        barmode='group',
        title='High Sales but Below Target Profit by State'
    )
    fig2a.show()
else:
    print("✅ All high-sales states are meeting or exceeding their profit targets.")


# --- [6] Calculate Profit Margin by State ---
df_state = df.groupby('State', as_index=False).agg({
    'Sales': 'sum',
    'Profit': 'sum'
})
df_state['Profit_Margin'] = df_state['Profit'] / df_state['Sales']
df_state = df_state.sort_values('Profit_Margin', ascending=False)


# --- [7] Visualize Profit Margins ---
fig2b = px.bar(
    df_state,
    x='State',
    y='Profit_Margin',
    title='Profit Margin by State',
    labels={'Profit_Margin': 'Profit Margin (%)'}
)
fig2b.update_layout(xaxis_tickangle=-45)
fig2b.show()


# --- [8] (Optional Insight) Total Expenses by State ---
expense_check = df.groupby('State', as_index=False)['Total_expenses'].sum()\
    .sort_values(by='Total_expenses', ascending=False)

print("\n💸 States with Highest Total Expenses:")
print(expense_check.head(10).to_string(index=False))


# --- [9] (Optional Insight) Total Marketing Spend by State ---
marketing_check = df.groupby('State', as_index=False)['Marketing'].sum()\
    .sort_values(by='Marketing', ascending=False)

print("\n📢 States with Highest Marketing Spend:")
print(marketing_check.head(10).to_string(index=False))


🚩 States with High Sales but Profit Below Target:
        State  Sales  Target_sales  Profit  Target_profit
     Colorado  12112         11120    4450           4600
  Connecticut   6670          5420    2066           2080
     Missouri   6072          5460     956           1660
New Hampshire   3543          2860     610           1040
   New Mexico   3805          3340     216            760
         Ohio   8787          8420    2766           3180
       Oregon  10661          9120    3309           3420
         Utah   9159          8240    2199           2940
   Washington   9744          8500    2865           2980
    Wisconsin   7839          7220    1937           2400



💸 States with Highest Total Expenses:
     State  Total_expenses
California            5642
    Nevada            4616
  New York            4312
  Illinois            3336
Washington            3212
    Oregon            3184
      Utah            3130
  Colorado            3060
      Iowa            2918
 Wisconsin            2834

📢 States with Highest Marketing Spend:
     State  Marketing
California       3802
  New York       3116
    Nevada       2938
  Illinois       2130
      Iowa       1832
Washington       1786
  Colorado       1700
 Wisconsin       1560
    Oregon       1554
   Florida       1480


## Question 2: Which products should we focus on?

In [7]:
# --- Step 1: Total Profit by Product ---
product_profit = df.groupby('Product', as_index=False).agg({
    'Profit': 'sum',
    'Product_line': 'first'
}).sort_values('Profit', ascending=False)

# Plot total profit by product
fig2 = px.bar(
    product_profit,
    x='Product',
    y='Profit',
    color='Product_line',
    color_discrete_map={
        'Beans': 'saddlebrown',
        'Leaves': 'forestgreen'
    },
    text='Profit',
    title='Total Profit by Product'
)
fig2.update_traces(texttemplate='%{text:.2f}', textposition='outside')
fig2.update_layout(
    xaxis={'categoryorder': 'total descending'},
    yaxis_title='Total Profit ($)',
    height=600
)
fig2.show()

# --- Step 2: Check for Products with Negative Profit ---
loss_makers = product_profit[product_profit['Profit'] < 0]

if loss_makers.empty:
    print("✅ No products have negative profit.")
else:
    print("⚠️ Products with Negative Profit:")
    print(loss_makers.to_string(index=False))

# --- Step 3: Profit by Product and State ---
state_product = df.groupby(['State', 'Product'], as_index=False).agg({
    'Sales': 'sum',
    'Profit': 'sum'
})

# Calculate percentage of total state sales per product
state_total_sales = df.groupby('State', as_index=False)['Sales'].sum()
state_total_sales.rename(columns={'Sales': 'Total_Sales'}, inplace=True)
merged = pd.merge(state_product, state_total_sales, on='State')
merged['Sales_Percentage'] = (merged['Sales'] / merged['Total_Sales']) * 100

# Sort by Sales descending
merged_sorted = merged.sort_values(['State', 'Sales'], ascending=[True, False])
top_products = merged_sorted.groupby('State').head(3).reset_index(drop=True)

print("\nTop 3 Products Sold per State:")
display(top_products)

# --- Step 4: Categorize Products by Profit Levels ---
bins = [-float('inf'), 0, 200, 600, 1200, float('inf')]
labels = ['Very Low', 'Low', 'Medium', 'High', 'Very High']

merged['Profit_Category'] = pd.cut(
    merged['Profit'],
    bins=bins,
    labels=labels
)

# --- Step 5: Merge Profit Category Back into State-Product Data ---
# Save updated version with Profit Category
category_distribution = merged.copy()

# Sort products within each state by descending Profit
merged_sorted_with_cat = category_distribution.sort_values(
    ['State', 'Profit'], ascending=[True, False]
)

print("\n📊 Product sales by Profit Category per State (sample):")
display(merged_sorted_with_cat.head(20))

# --- Step 6: Sort States by total Sales_Percentage descending ---
state_totals = category_distribution.groupby('State')['Sales_Percentage'].sum().sort_values(ascending=False)
sorted_states = state_totals.index.tolist()

# Update State to be a categorical column with correct sort order
category_distribution['State'] = pd.Categorical(category_distribution['State'], categories=sorted_states, ordered=True)

fig2a = px.bar(
    category_distribution,
    x='State',
    y='Sales_Percentage',
    color='Profit_Category',
    hover_data=['Product'],  # ✅ Add product name to hover
    category_orders={'Profit_Category': labels, 'State': sorted_states},
    title='Product Sales by Profit Category per State (Sorted by Total Sales %)',
    labels={'Sales_Percentage': '% of Sales'}
)

fig2a.update_layout(
    xaxis={'categoryorder': 'array', 'categoryarray': sorted_states},
    height=600
)
fig2a.show()

# --- Step 7: Drill-down style comparison ---
states_available = category_distribution['State'].unique().tolist()

def plot_comparison(state_x, state_y):
    subset = merged_sorted_with_cat[
        merged_sorted_with_cat['State'].isin([state_x, state_y])
    ]
    comparison_df = subset.groupby(['State', 'Profit_Category'], observed=True).agg({'Sales_Percentage':'sum'}).reset_index()
    comparison_df['Profit_Category'] = pd.Categorical(comparison_df['Profit_Category'], categories=labels, ordered=True)

    fig2b = px.bar(
        comparison_df,
        x='Profit_Category',
        y='Sales_Percentage',
        color='State',
        barmode='group',
        category_orders={'Profit_Category': labels},
        title=f'Sales % by Profit Category: {state_x} vs {state_y}',
        labels={'Sales_Percentage': 'Sales %', 'Profit_Category': 'Profit Category'}
    )
    fig2b.update_layout(yaxis=dict(range=[0, 100]))
    fig2b.show()

# Example default comparison
plot_comparison('California', 'Illinois')
# --- Step 8: State-Level Product Sales Ranked by Product Profitability ---

# Filter for California and Illinois
df_states = df[df['State'].isin(['California', 'Illinois'])]

# Step A: Get total profit per product across all states (used for ranking)
product_profit_total = df.groupby('Product', as_index=False)['Profit'].sum()

# Step B: Get total sales per product per state
product_sales_by_state = df_states.groupby(['Product', 'State'], as_index=False)['Sales'].sum()

# Step C: Merge total profit into the sales-by-state data
merged = pd.merge(product_sales_by_state, product_profit_total, on='Product', how='left')

# Step D: Rank products by total profit (higher profit = higher rank)
merged['Profit_Rank'] = merged.groupby('Product')['Profit'].transform('max')  # In case of duplicates
merged = merged.sort_values(by='Profit', ascending=False)
merged['Profit_Rank'] = merged['Profit'].rank(method='dense', ascending=False).astype(int)

# Optional: Limit to top N most profitable products
top_n = 12
top_products = merged[merged['Profit_Rank'] <= top_n]

# Step E: Plot Sales by Product per State (ranked by profitability)
fig2c = px.bar(
    top_products,
    x='Profit_Rank',
    y='Sales',
    color='State',
    text='Product',
    barmode='group',
    title='Product Sales by State (Ranked by Total Profitability)',
    labels={'Profit_Rank': 'Profit Rank (1 = Most Profitable)', 'Sales': 'Total Sales ($)'}
)

fig2c.update_layout(
    xaxis=dict(tickmode='linear', dtick=1),
    height=600
)
fig2c.update_traces(textposition='outside')
fig2c.show()

✅ No products have negative profit.

Top 3 Products Sold per State:


,State,Product,Sales,Profit,Total_Sales,Sales_Percentage
0,California,Colombian,4074,1871,23032,17.688433
1,California,Decaf Espresso,3524,1585,23032,15.300452
2,California,Lemon,3170,1361,23032,13.763460
3,Colorado,Chamomile,2047,711,12112,16.900594
4,Colorado,Amaretto,1847,981,12112,15.249339
5,Colorado,Decaf Irish Cream,1390,716,12112,11.476222
6,Connecticut,Colombian,1915,850,6670,28.710645
7,Connecticut,Lemon,1062,245,6670,15.922039
8,Connecticut,Mint,890,270,6670,13.343328
9,Florida,Decaf Irish Cream,1617,713,8825,18.322946



📊 Product sales by Profit Category per State (sample):


,State,Product,Sales,Profit,Total_Sales,Sales_Percentage,Profit_Category
4,California,Colombian,4074,1871,23032,17.688433,Very High
6,California,Decaf Espresso,3524,1585,23032,15.300452,Very High
10,California,Lemon,3170,1361,23032,13.763460,Very High
1,California,Caffe Latte,2896,1080,23032,12.573810,High
5,California,Darjeeling,1847,984,23032,8.019278,High
3,California,Chamomile,1390,716,23032,6.035082,High
8,California,Earl Grey,915,445,23032,3.972734,Medium
9,California,Green Tea,1113,403,23032,4.832407,Medium
2,California,Caffe Mocha,1878,209,23032,8.153873,Medium
11,California,Mint,581,204,23032,2.522577,Medium


# Question 3: Are our markets meeting their goals?

In [8]:
# === [1] Market & State Profit vs Target Analysis ===

# Group by Market and State
market_perf = df.groupby('Market', as_index=False).agg({
    'Profit': 'sum',
    'Target_profit': 'sum',
    'DifferenceBetweenActualandTargetProfit': 'sum'
})
state_perf = df.groupby('State', as_index=False).agg({
    'Profit': 'sum',
    'Target_profit': 'sum',
    'DifferenceBetweenActualandTargetProfit': 'sum'
})

# Flag whether quota was met
market_perf['Met_Quota'] = market_perf['DifferenceBetweenActualandTargetProfit'] > 0
state_perf['Met_Quota'] = state_perf['DifferenceBetweenActualandTargetProfit'] > 0

# Sort by performance
market_perf = market_perf.sort_values('DifferenceBetweenActualandTargetProfit', ascending=False)
state_perf = state_perf.sort_values('DifferenceBetweenActualandTargetProfit', ascending=False)

# Plot: Market vs State Performance
fig3 = make_subplots(
    rows=1, cols=2,
    subplot_titles=["Are Our States Meeting Their Profit Goals?", "Are Our Markets Meeting Their Profit Goals?"]
)

# Add State performance bars
fig3.add_trace(
    go.Bar(
        x=state_perf['State'],
        y=state_perf['DifferenceBetweenActualandTargetProfit'],
        name='State Profit vs Target',
        marker_color=state_perf['Met_Quota'].map({True: 'green', False: 'red'}),
        text=state_perf['DifferenceBetweenActualandTargetProfit'].round(2),
        hovertext=[
            f"Profit: ${p:,.0f}<br>Target: ${t:,.0f}<br>Diff: ${d:,.0f}"
            for p, t, d in zip(
                state_perf['Profit'],
                state_perf['Target_profit'],
                state_perf['DifferenceBetweenActualandTargetProfit']
            )
        ],
        hoverinfo="text"
    ),
    row=1, col=1
)

# Add Market performance bars
fig3.add_trace(
    go.Bar(
        x=market_perf['Market'],
        y=market_perf['DifferenceBetweenActualandTargetProfit'],
        name='Market Profit vs Target',
        marker_color=market_perf['Met_Quota'].map({True: 'green', False: 'red'}),
        text=market_perf['DifferenceBetweenActualandTargetProfit'].round(2),
        hovertext=[
            f"Profit: ${p:,.0f}<br>Target: ${t:,.0f}<br>Diff: ${d:,.0f}"
            for p, t, d in zip(
                market_perf['Profit'],
                market_perf['Target_profit'],
                market_perf['DifferenceBetweenActualandTargetProfit']
            )
        ],
        hoverinfo="text"
    ),
    row=1, col=2
)

# Add formatting and layout
fig3.update_layout(
    height=500,
    width=1000,
    showlegend=False,
    xaxis_title='State',
    yaxis_title='Profit vs Target (Difference)',
    xaxis2_title='Market',
    yaxis2_title='Profit vs Target (Difference)',
    yaxis_tickprefix='$',
    yaxis2_tickprefix='$',
    xaxis={'categoryorder': 'total descending'},
    xaxis2={'categoryorder': 'total descending'}
)

# Add baseline reference lines (y=0)
fig3.add_shape(
    type='line',
    x0=-0.5, x1=len(state_perf) - 0.5,
    y0=0, y1=0,
    line=dict(color='black', width=1, dash='dash'),
    xref='x', yref='y'
)
fig3.add_shape(
    type='line',
    x0=-0.5, x1=len(market_perf) - 0.5,
    y0=0, y1=0,
    line=dict(color='black', width=1, dash='dash'),
    xref='x2', yref='y2'
)

fig3.show()


# === [2] Sales vs Profit by State (Grouped Bar Chart) ===

state_summary = df.groupby('State', as_index=False)[['Sales', 'Profit']].sum()
state_summary = state_summary.sort_values('Sales', ascending=False)

fig3a = go.Figure(data=[
    go.Bar(name='Sales', x=state_summary['State'], y=state_summary['Sales']),
    go.Bar(name='Profit', x=state_summary['State'], y=state_summary['Profit'])
])
fig3a.update_layout(
    barmode='group',
    title='Sales vs Profit by State (Grouped Bar Chart)',
    xaxis_title='State',
    yaxis_title='Amount ($)',
    xaxis_tickangle=-45,
    height=500,
    width=1000
)
fig3a.show()


# === [3] Compare Actual vs Target Sales & Profit ===

# Calculate differences
df['Profit_vs_Target'] = df['Profit'] - df['Target_profit']
df['Sales_vs_Target'] = df['Sales'] - df['Target_sales']


# === [4] Aggregate State Performance vs Target ===

state_perf = df.groupby('State', as_index=False)[[
    'Sales', 'Target_sales', 'Sales_vs_Target',
    'Profit', 'Target_profit', 'Profit_vs_Target'
]].sum()


# === [5] Identify Underperforming States ===

# Criteria: High sales but profit below target
underperforming_states = state_perf[
    (state_perf['Sales'] > state_perf['Target_sales']) &
    (state_perf['Profit'] < state_perf['Target_profit'])
]

print("🚩 States with High Sales but Profit Below Target:")
print(underperforming_states[['State', 'Sales', 'Target_sales', 'Profit', 'Target_profit']].to_string(index=False))


# === [6] Visualize Underperforming States ===

if not underperforming_states.empty:
    fig3b = px.bar(
        underperforming_states,
        x='State',
        y=['Sales', 'Target_sales', 'Profit', 'Target_profit'],
        barmode='group',
        title='High Sales but Below Target Profit by State'
    )
    fig3b.show()
else:
    print("✅ All high-sales states are meeting or exceeding their profit targets.")


# === [7] Profit Margin Analysis by State ===

# Calculate margin
df_state = df.groupby('State', as_index=False).agg({
    'Sales': 'sum',
    'Profit': 'sum'
})
df_state['Profit_Margin'] = df_state['Profit'] / df_state['Sales']
df_state = df_state.sort_values('Profit_Margin', ascending=False)

# Plot
fig3c = px.bar(
    df_state,
    x='State',
    y='Profit_Margin',
    title='Profit Margin by State',
    labels={'Profit_Margin': 'Profit Margin (%)'}
)
fig3c.update_layout(xaxis_tickangle=-45)
fig3c.show()


# === [8] (Optional Insight) Top States by Total Expenses ===

expense_check = df.groupby('State', as_index=False)['Total_expenses'].sum()\
    .sort_values(by='Total_expenses', ascending=False)

print("\n💸 States with Highest Total Expenses:")
print(expense_check.head(10).to_string(index=False))


# === [9] (Optional Insight) Top States by Marketing Spend ===

marketing_check = df.groupby('State', as_index=False)['Marketing'].sum()\
    .sort_values(by='Marketing', ascending=False)

print("\n📢 States with Highest Marketing Spend:")
print(marketing_check.head(10).to_string(index=False))

🚩 States with High Sales but Profit Below Target:
        State  Sales  Target_sales  Profit  Target_profit
     Colorado  12112         11120    4450           4600
  Connecticut   6670          5420    2066           2080
     Missouri   6072          5460     956           1660
New Hampshire   3543          2860     610           1040
   New Mexico   3805          3340     216            760
         Ohio   8787          8420    2766           3180
       Oregon  10661          9120    3309           3420
         Utah   9159          8240    2199           2940
   Washington   9744          8500    2865           2980
    Wisconsin   7839          7220    1937           2400



💸 States with Highest Total Expenses:
     State  Total_expenses
California            5642
    Nevada            4616
  New York            4312
  Illinois            3336
Washington            3212
    Oregon            3184
      Utah            3130
  Colorado            3060
      Iowa            2918
 Wisconsin            2834

📢 States with Highest Marketing Spend:
     State  Marketing
California       3802
  New York       3116
    Nevada       2938
  Illinois       2130
      Iowa       1832
Washington       1786
  Colorado       1700
 Wisconsin       1560
    Oregon       1554
   Florida       1480


# 📌 Conclusions

---

## 🗺️ Market-Level Performance

* ✅ **East** Market leads with a **profit surplus of \$1,165**, driven by strong contributions from **New York** and **Massachusetts**.
* ✅ **West** Market shows a modest **profit gain of \$149**, though performance is mixed across states.
* ⚠️ **South** (-\$244) and **Central** (-\$659) Markets **missed their profit targets**, with **underperformance concentrated in states like Texas, Missouri, Utah, and New Mexico**.

---

## 📍 State-Level Insights

### 🔝 Top-Performing States:

| State          | Profit Above Target | Key Drivers                        |
| -------------- | ------------------- | ---------------------------------- |
| **Iowa**       | +\$1,152            | High-margin products (Colombian)   |
| **New York**   | +\$925              | Strong sales despite high expenses |
| **California** | +\$860              | Balanced sales and margin strategy |

### 🔻 Underperforming States:

| State          | Profit Miss | Margin   | Key Issues                                  |
| -------------- | ----------- | -------- | ------------------------------------------- |
| **Utah**       | -\$741      | 24%      | High expenses, Decaf Espresso (low margin)  |
| **Missouri**   | -\$574      | 16%      | Poor mix: Chamomile + Green Tea             |
| **New Mexico** | -\$544      | **6.7%** | Chamomile (7.17%) + Decaf Espresso (14.27%) |
| **Wisconsin**  | -\$463      | 29%      | High marketing + low-margin product mix     |

---

## ☕ Product-Level Insights

### 📈 High-Margin Products:

| Product              | Profit Margin (%) |
| -------------------- | ----------------- |
| **Regular Espresso** | 24.43%            |
| **Darjeeling**       | 23.78%            |
| **Caffe Mocha**      | \~23%             |

➡️ Often **underrepresented in top-selling states** — potential to promote more.

### ⚠️ Low-Margin Product Concentration:

| Product            | Margin (%)   | Problem States |
| ------------------ | ------------ | -------------- |
| **Chamomile**      | 7.17%–10.72% | NM, WI, MO     |
| **Decaf Espresso** | 7.99%–14.27% | UT, NM, OR     |
| **Green Tea**      | 7.54%        | NH, MO, OR     |

---

## 🔎 Underperforming Product Combinations

| State          | Key Products              | Observations                                       |
| -------------- | ------------------------- | -------------------------------------------------- |
| **New Mexico** | Chamomile, Decaf Espresso | Lowest profit margin and volume                    |
| **Missouri**   | Green Tea, Chamomile      | Weak margins and low profitability                 |
| **Utah**       | Decaf Espresso            | Underperforms despite decent sales                 |
| **Wisconsin**  | Chamomile, Caffe Mocha    | High expenses, marketing not translating to profit |

---

## 📊 Profit Margin vs Volume

* 💰 High-margin products like **Darjeeling** and **Colombian Coffee** are **underutilized** across states.
* 🍋 **Lemon** (11%–21%) and **Chamomile** are frequently sold but yield **low profits**.
* Example:

  * **Nevada** and **New York** sell large volumes of **Lemon**, but profit margins remain moderate.

➡️ **Recommendation**: Increase visibility and marketing of **high-margin, under-leveraged products** in key states.

---

## 🧠 Strategy Recommendations

### 📍 Focus Areas by Market

* **East**: Strong performance but reliant on few states — **expand success** with broader margin-rich product adoption.
* **West**: Mixed results — address margin gaps in **New Mexico** and **Utah**, support growth in **Colorado**.
* **South & Central**: Combined shortfall of **\$903**:

  * Optimize **product mix in Texas** (high sales, moderate margin).
  * Address poor mix and high costs in **Missouri** and **Utah**.

---